### **Time Series Classification: Chinatown**

DATASET: https://www.timeseriesclassification.com/description.php?Dataset=Chinatown

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install gpjax
!pip install sktime

In [ ]:
# jax imports
import numpy as np
import jax
import jax.numpy as jnp
from jax import random

# gpjax imports
import gpjax as gpx
from gpjax.linalg import Dense, psd
from gpjax.parameters import Parameter, NonNegativeReal, Real, LowerTriangular

# low level imports
from flax import nnx
import optax as ox

# formatting imports
import pandas as pd
import sktime
from sktime.datasets import load_from_tsfile
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
!rm -rf /content/AlexanderAdamsMastersThesis

!git clone https://github.com/alexandertaoadams/AlexanderAdamsMastersThesis.git

from AlexanderAdamsMastersThesis.kernels import SignatureKernel, CustomComputeEngine
from AlexanderAdamsMastersThesis.algorithms import Gram_XX_jit, Cross_XZ_jit, Diag_XX_jit
from AlexanderAdamsMastersThesis.inducing_variables import initial_inducing_variables
from AlexanderAdamsMastersThesis.utils import display_parameters, display_results

from AlexanderAdamsMastersThesis.models import CollapsedBernoulliVariationalGaussian
from AlexanderAdamsMastersThesis.objectives import collapsed_elbo_bernoulli

### **Data Loading and Preprocessing**

In [ ]:
def normalise(data_as_array, eps = 1e-12):
    mean = jnp.mean(data_as_array, axis=0)
    std = jnp.std(data_as_array, axis=0)
    normalised_data = (data_as_array - mean)/(std + eps)
    return normalised_data, mean, std

In [ ]:
file_path_train = "/content/drive/MyDrive/DATA_Chinatown/Chinatown_TRAIN.ts"
file_path_test = "/content/drive/MyDrive/DATA_Chinatown/Chinatown_TEST.ts"

train_data, train_labels = load_from_tsfile(file_path_train)
test_data, test_labels = load_from_tsfile(file_path_test)

In [ ]:
train_data_2 = jnp.array((np.stack([np.stack(row) for row in train_data.to_numpy()])))
train_labels_2 = jnp.array([int(i)-1 for i in train_labels])
xtrain, train_mean, train_std = normalise(train_data_2)
ytrain = train_labels_2
print(xtrain.shape)
print(ytrain.shape)
print(ytrain)

In [ ]:
test_data_2 = jnp.array((np.stack([np.stack(row) for row in test_data.to_numpy()])))
test_labels_2 = jnp.array([int(i)-1 for i in test_labels])
xtest = (test_data_2 - train_mean) / (train_std)
ytest = test_labels_2
print(xtest.shape)
print(ytest.shape)
print(ytest)

### **Model Training**



In [ ]:
# Initialising model
q_kernel = SignatureKernel(xtrain.shape[1], xtrain.shape[2], 4)
q_mean_function = gpx.mean_functions.Constant(constant=jnp.array([0.5]))
q_prior = gpx.gps.Prior(mean_function=q_mean_function, kernel=q_kernel)
q_likelihood = gpx.likelihoods.Bernoulli(xtrain.shape[0])
q_posterior = q_likelihood * q_prior

# Create dataset, initialise inducing variables
D = gpx.dataset.Dataset(jnp.reshape(xtrain, (xtrain.shape[0], -1)), jnp.expand_dims(ytrain, axis=1))
Z = initial_inducing_variables(xtrain, ytrain, 8)

# Model
q_collapsed = CollapsedBernoulliVariationalGaussian(
    posterior=q_posterior,
    inducing_inputs=Z
)

In [ ]:
# Training
collapsed_optimised_model, history = gpx.fit(
    model=q_collapsed,
    objective= lambda model, data: -collapsed_elbo_bernoulli(model, data),
    train_data=D,
    optim=ox.adam(learning_rate=1e-4),
    trainable=Parameter,
    num_iters=2000,
    batch_size=20,
    verbose=True
)

In [ ]:
# Plot training process
sns.set(style="whitegrid", context="talk")
plt.figure(figsize=(8, 5))
plt.plot(history, color="black", linewidth=0.5, label="Training ELBO")
plt.xlabel("Iteration", fontsize=12)
plt.ylabel("−ELBO", fontsize=12)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.tight_layout()
plt.show()

In [ ]:
# Display model hyperparameters
params = display_parameters(optimised_model)
params

### **Model Evaluation**

In [ ]:
# Get predicted means
latent_dist = collapsed_optimised_model.predict(xtest, D)
predicted_dist = collapsed_optimised_model.posterior.likelihood(latent_dist)
predicted_mean_collapsed = predicted_dist.mean

# Get predicted class labels (round to nearest int)
predicted_labels_collapsed = jnp.round(predicted_mean_collapsed)

# print("Predicted means:", predicted_mean_collapsed)
# print("Predicted labels:", predicted_labels_collapsed)
# print("True labels:", ytest)


In [ ]:
# Display results
results = display_results(predicted_labels_collapsed, ytest)
results